Hacker.ipynb
============

This notebook attempts to use my langchain alternatives to hack a target device. The goal of the notebook is to either show the agent working, or to shine a light on what aspects need to be improved. The notebook should be continuously updated to use the latest improvemenets in the agent.


In [1]:
import os
import sys
import time
import openai

from dotenv import load_dotenv

from agent import Agent
from tasker import TaskManager
from entityDB import EntityDatabase
from shelltool import ExecutionSandbox

load_dotenv()
openai.organization = os.getenv("OPENAI_ORGANIZATION")
openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
# Just ensure that things are properly killed
os.system("podman kill rat-machine-sandbox")
os.system("podman rm rat-machine-sandbox")
time.sleep(1)

Error: no container with name or ID "rat-machine-sandbox" found: no such container
Error: no container with ID or name "rat-machine-sandbox" found: no such container


In [3]:
task_manager = TaskManager(openai)
database = EntityDatabase(openai, "gpt-3.5-turbo")
agent = Agent(tools=[ExecutionSandbox()], verbose=False)

In [4]:
while True:
    task = task_manager.get_next()

    # ran out of tasks
    if task == "":
        break
        
    
    print("--------------context---------------")
    print(database.database)
    print("------------------------------------")
        
    context = ""
    if database.database != "":
        context = database.get_context(task)


    print("--------------context---------------")
    print(f"Context: {context}")
    print(f"Task: {task}")
    print("------------------------------------")

    
    output = agent.run(task, context=context)
    database.update(output)
    task_manager.mark_next_completed()

    print("--------------output----------------")
    print(output)
    print("------------------------------------\n\n")

--------------context---------------

------------------------------------
--------------context---------------
Context: 
Task: Use `ip a` to find me your ip address and the local IP range.
------------------------------------
--------------output----------------
The local IP address for the machine is 192.168.1.179 and the local IP range is 192.168.1.0 to 192.168.1.255. Additionally, for the virtual bridge interface `virbr0`, the IP address is `192.168.122.1` with the range from `192.168.122.0` to `192.168.122.255`.
------------------------------------


--------------context---------------
192.168.1.179: The machine has a local IP address of 192.168.1.179 and is part of the local IP range 192.168.1.0 to 192.168.1.255.
------------------------------------
--------------context---------------
Context: 192.168.1.179 is within the local IP range of 192.168.1.0 to 192.168.1.255.
Task: Find all connected on the above mentioned range
------------------------------------
--------------output